In [ ]:
import os
import sys
import sqlite3
import json
import tempfile
import spider.evaluation as sp
from dotenv import load_dotenv
load_dotenv()

ROOT_PATH = os.environ.get("ROOT_PATH")
TMP_DIR = os.environ.get("TMP_DIR")

if not ROOT_PATH:
    raise ValueError("ROOT_PATH environment variable not set. Please set it in your .env file.")
if not TMP_DIR:
    raise ValueError("TMP_DIR environment variable not set. Please set it in your .env file.")

BRONZE_DB = f"{ROOT_PATH}/database/bronze/bronze.sqlite"

conn_bronze = sqlite3.connect(BRONZE_DB)
conn_bronze.row_factory = sqlite3.Row

cursor_bronze = conn_bronze.cursor()
cursor_bronze.execute("SELECT * FROM spider_tables")
rows = cursor_bronze.fetchall()

json_fields = ['table_names', 'table_names_original', 'column_names', 
               'column_names_original', 'column_types', 'primary_keys', 'foreign_keys']

json_data = []
for row in rows:
    row_dict = dict(row)
    
    for field in json_fields:
        if field in row_dict and row_dict[field]:
            try:
                row_dict[field] = json.loads(row_dict[field])
            except json.JSONDecodeError:
                pass
    
    json_data.append(row_dict)

json_string = json.dumps(json_data, indent=2)

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as temp_file:
    temp_file.write(json_string)
    temp_file_path = temp_file.name

KMAPS = sp.build_foreign_key_map_from_json(temp_file_path)
os.unlink(temp_file_path)

PREDICTED_DATA = f'{ROOT_PATH}/data'
DB_DIR = f"{ROOT_PATH}/database/spider"
GOLD = f"{ROOT_PATH}/data/training/nl2SQL/template_11/t_valid.sql"
PREDICT = f"{PREDICTED_DATA}/predictions/pred.sql"
ETYPE = "all" # all, easy, medium, hard

with open('output.txt', 'w') as f:
    original_stdout = sys.stdout
    sys.stdout = f
    sp.evaluate(gold=GOLD , predict=PREDICT, db_dir=DB_DIR, etype=ETYPE, kmaps=KMAPS)
    sys.stdout = original_stdout
